In [ ]:
if True:
    %matplotlib notebook
elif False:
    %matplotlib inline
else:
    %matplotlib qt

In [ ]:
import getpass

import pyelegant as pe
from pyelegant import nonlin
from pyelegant import util

In [ ]:
pe.disable_stdout()

if True:
    remote_mail_notifications = {}
else:
    username = getpass.getuser()
    remote_mail_notifications = dict(
        mail_type='END',
        mail_user=f'{username}@bnl.gov',
    )

In [ ]:
LTE_filepath = 'nsls2cb65pm.lte'
use_beamline = 'RING' # Must be the beamline for a full ring, NOT one cell
E_MeV = 3e3

In [ ]:
%%timeit -n 1 -r 1
# 20min 39s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each) on nsls2apcluster
# 29min 45s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each) on nsls2pluto

# Example for RF/radiation both off & for the full ring
if False:

    n_turns = 128
    s_start = 0.0
    s_end = None # full ring, NOT just one super cell
    include_name_pattern = '[QS]*'
    x_initial = 1e-5
    y_initial = 1e-5
    delta_negative_start = -1e-3
    delta_positive_start = +1e-3
    delta_negative_limit = -5e-2
    delta_positive_limit = +5e-2
    init_delta_step_size = 5e-3    

    ntasks = 100

    remote_opts = dict(
        pelegant=True, job_name='momaper', ntasks=ntasks,
        sbatch={'use': True, 'wait': True},
    )
    remote_opts.update(remote_mail_notifications)
    if pe.facility_name == 'nsls2apcluster':
        remote_opts['partition'] = 'short'
        remote_opts['time'] = '1:00:00'
        #remote_opts['nodelist'] = ['apcpu-004']
    elif pe.facility_name == 'nsls2pluto':
        remote_opts['partition'] = 'normal'
        remote_opts['qos'] = 'normal'
        remote_opts['time'] = '1:00:00'
        #remote_opts['nodelist'] = ['hpc005']

    output_filepath = f'test_momAper_n{n_turns}.hdf5'
    nonlin.calc_mom_aper(
        output_filepath, LTE_filepath, E_MeV, 
        use_beamline=use_beamline, # Must be the beamline for a full ring, NOT one cell
        x_initial=x_initial, y_initial=y_initial,
        delta_negative_start=delta_negative_start, delta_negative_limit=delta_negative_limit,
        delta_positive_start=delta_positive_start, delta_positive_limit=delta_positive_limit,
        init_delta_step_size=init_delta_step_size, s_start=s_start, s_end=s_end,
        include_name_pattern=include_name_pattern, n_turns=n_turns, 
        rf_cavity_on=False, radiation_on=False,
        run_local=False, remote_opts=remote_opts)


In [ ]:
output_filepath = 'test_momAper_n128.hdf5'

title = f'$\mathrm{{{LTE_filepath}}}$'

nonlin.plot_mom_aper(output_filepath, title=title, slim=[0, 792/15])

In [ ]:
RFharm = 1320
RFvolt = 3e6

twi_d = pe.util.load_sdds_hdf5_file(output_filepath)[0]['twi']
circumf = twi_d['arrays']['s'][-1]
print(f'Circumference [m]: {circumf:.6g}')
U0_eV = twi_d['scalars']['U0'] * 1e6

rf_params = nonlin.calc_ring_rf_params(RFharm, circumf, U0_eV, rf_volt=RFvolt)
rf_params

In [ ]:
# Need to create an SDDS file from the HDF5 file, which can be directly fed
# into the ELEGANT's lifetime calculation function.

mmap_h5_filepath = 'test_momAper_n128.hdf5' # This is for the furll ring
mmap_sdds_filepath = mmap_h5_filepath.replace('.hdf5', '.mmap')
mmap_d = pe.util.load_sdds_hdf5_file(mmap_h5_filepath)[0]['mmap']
pe.sdds.dicts2sdds(mmap_sdds_filepath, params=mmap_d['scalars'],
                   columns=mmap_d['arrays'], outputMode='binary')

In [ ]:
Ib = 0.5e-3 # [A]
rf_freq_Hz = rf_params['freq_Hz']
charge_C = Ib / (rf_freq_Hz / RFharm)
emit_ratio = 1.0
max_mom_aper_percent = None
ignoreMismatch = True

print_cmd = True

In [ ]:
output_filepath = 'test_tau.pgz'
nonlin.calc_Touschek_lifetime(
    output_filepath, LTE_filepath, E_MeV, mmap_sdds_filepath, charge_C, emit_ratio, RFvolt, RFharm, 
    use_beamline=use_beamline, # Must be the beamline for a full ring, NOT one cell
    max_mom_aper_percent=max_mom_aper_percent, 
    ignoreMismatch=ignoreMismatch, print_cmd=print_cmd)

In [ ]:
d = pe.util.load_pgz_file(output_filepath)
d['data']['life']

In [ ]:
title = f'$\mathrm{{{LTE_filepath}}}$'

nonlin.plot_Touschek_lifetime(output_filepath, title=title)

In [ ]:
circumf = 791.958

In [ ]:
%%timeit -n 1 -r 1
# 4min 46s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each) on nsls2pluto

# Example for RF/radiation both off & for one super cell
if False:

    n_turns = 128
    s_start = 0.0
    s_end = circumf / 15 # Just one super cell, NOT a full ring
    # (Note that "use_beamline" should still be a beamline for a full ring, as the Twiss parameters
    # used must be for the full ring.)
    include_name_pattern = '[QS]*'
    x_initial = 1e-5
    y_initial = 1e-5
    delta_negative_start = -1e-3
    delta_positive_start = +1e-3
    delta_negative_limit = -5e-2
    delta_positive_limit = +5e-2
    init_delta_step_size = 5e-3    

    ntasks = 100

    remote_opts = dict(
        pelegant=True, job_name='momaper', ntasks=ntasks,
        sbatch={'use': True, 'wait': True},
    )
    remote_opts.update(remote_mail_notifications)
    if pe.facility_name == 'nsls2apcluster':
        remote_opts['partition'] = 'short'
        remote_opts['time'] = '1:00:00'
        #remote_opts['nodelist'] = ['apcpu-004']
    elif pe.facility_name == 'nsls2pluto':
        remote_opts['partition'] = 'normal'
        remote_opts['qos'] = 'normal'
        remote_opts['time'] = '1:00:00'
        #remote_opts['nodelist'] = ['hpc005']

    output_filepath = f'test_momAper_oneSupCell_n{n_turns}.hdf5'
    nonlin.calc_mom_aper(
        output_filepath, LTE_filepath, E_MeV, 
        use_beamline=use_beamline, # Must be the beamline for a full ring, NOT one cell
        x_initial=x_initial, y_initial=y_initial,
        delta_negative_start=delta_negative_start, delta_negative_limit=delta_negative_limit,
        delta_positive_start=delta_positive_start, delta_positive_limit=delta_positive_limit,
        init_delta_step_size=init_delta_step_size, s_start=s_start, s_end=s_end,
        include_name_pattern=include_name_pattern, n_turns=n_turns, 
        rf_cavity_on=False, radiation_on=False,
        run_local=False, remote_opts=remote_opts)

In [ ]:
output_filepath = 'test_momAper_oneSupCell_n128.hdf5'

title = f'$\mathrm{{{LTE_filepath}}}$'

nonlin.plot_mom_aper(output_filepath, title=title)

In [ ]:
RFharm = 1320
RFvolt = 3e6

# This Twiss data should be for the full ring, NOT one super-cell.
twi_d = pe.util.load_sdds_hdf5_file(output_filepath)[0]['twi']
circumf = twi_d['arrays']['s'][-1]
print(f'Circumference [m]: {circumf:.6g}')
U0_eV = twi_d['scalars']['U0'] * 1e6

rf_params = nonlin.calc_ring_rf_params(RFharm, circumf, U0_eV, rf_volt=RFvolt)
rf_params

In [ ]:
# Since the momentum aperture data only extend to one super period,
# the data must be duplicated to cover the whole ring. Also, create
# an SDDS file from the HDF5 file, which can be directly fed
# into the ELEGANT's lifetime calculation function.

mmap_h5_filepath = 'test_momAper_oneSupCell_n128.hdf5' # This is for one super-cell
mmap_sdds_filepath_cell = mmap_h5_filepath.replace('.hdf5', '.mmap')
mmap_sdds_filepath_ring = mmap_h5_filepath.replace('.hdf5', '.mmapxt')
mmap_d = pe.util.load_sdds_hdf5_file(mmap_h5_filepath)[0]['mmap']

# First generate a mom-aper SDDS file for one super-cell
pe.sdds.dicts2sdds(mmap_sdds_filepath_cell, params=mmap_d['scalars'],
                   columns=mmap_d['arrays'], outputMode='binary')

# Then extend the mom-aper data to the full ring by duplicating
n_periods_in_ring = 15
dup_filenames = ' '.join(
    [mmap_sdds_filepath_cell] * n_periods_in_ring)
msectors = 1
cmd_list = [
    f'sddscombine {dup_filenames} -pipe=out',
    f'sddsprocess -pipe "-redefine=col,s,s i_page 1 - {circumf:.16g} {n_periods_in_ring} / * {msectors} * +,units=m"',
    f'sddscombine -pipe -merge',
    f'sddsprocess -pipe=in {mmap_sdds_filepath_ring} -filter=col,s,0,{circumf:.16g}',
]
if False: print(cmd_list)
result, err, returncode = pe.util.chained_Popen(cmd_list)
result, err, returncode

In [ ]:
Ib = 0.5e-3 # [A]
rf_freq_Hz = rf_params['freq_Hz']
charge_C = Ib / (rf_freq_Hz / RFharm)
emit_ratio = 1.0
max_mom_aper_percent = None
ignoreMismatch = True

print_cmd = True

In [ ]:
output_filepath = 'test_tau_oneSupCell.pgz'
nonlin.calc_Touschek_lifetime(
    output_filepath, LTE_filepath, E_MeV, mmap_sdds_filepath_ring, charge_C, emit_ratio, RFvolt, RFharm, 
    use_beamline=use_beamline, # Must be the beamline for a full ring, NOT one cell
    max_mom_aper_percent=max_mom_aper_percent, 
    ignoreMismatch=ignoreMismatch, print_cmd=print_cmd)

In [ ]:
d = pe.util.load_pgz_file(output_filepath)
d['data']['life']

In [ ]:
title = f'$\mathrm{{{LTE_filepath}}}$'

nonlin.plot_Touschek_lifetime(output_filepath, title=title)